In [1]:
import numpy as np
import pandas as pd
import sklearn
import scipy as s
from IPython.display import display
import matplotlib.pyplot as plt

# Table of Contents

# Introduction

Ranking systems have many different applications, however most common ones are poor as they rely on arbitrary conventions which lead to poor proformance. In general, they try to answer they question **What is the probability that player 1 defeats player 2?**. In order to determine this probabilistically, there are a number of things to consider:

- Considers who you played against.
- Must be robust against players who have not played against eachother.
- Give a good estimate at any point in the season.
- Take into account performance inconsistancy.

Therefore, what we want to infer is the player's **skill**, $w$. These skills must be comparable (i.e. a player with a higher skill must be more likely to win), and as such we want to do a probabilstic inference of a player's skill and be able to compute the probability of a game's outcome.

### A generative model for skill

A summary of a generative model for game outcomes can be defined as:

1. **Skills** Take two players with known skills, $$w_i \in \mathbb{R}$$
2. **Skill Difference**: $$s = w_1 - w_2$$
3. **Performance Difference**: Add noise ($n \sim \mathcal{N}(0, 1))$ to account for performance inconsistance: $$t = s + n$$
4. **Game outcome** is given by $y=sign(t)$:
    - $y = +1$ means player 1 wins
    - $y = -1$ means player 2 wins

![FactorGraph](Figures/FactorGraph.jpg)

### The probability of a player winning given their skills
    
Therefore we can work out the probability that player 1 wins given the players skills':

$$p(y| w_1, w_2) = \int \int p(y|t) p(t|s) p(s | w_1, w_2) $$
$$ = \int p(y|t) p(t|w_1, w_2) dt$$
$$ = \int^{+\infty}_{-\infty} \delta(y - sign(t)) \mathcal{N}(t; w_1 - w_2, 1) dt$$
$$ = \int^{+\infty}_{-\infty} \delta(1 - sign(yt)) \mathcal{N}(yt; y(w_1 - w_2), 1) dt $$
set z = yt and note the change in limits and dt:
$$ = \int^{+y\infty}_{-y\infty} \delta(1 - sign(z)) \mathcal{N}(z; y(w_1 - w_2), 1) y dz $$
$$ = \int^{+\infty}_{-\infty} \delta(1 -sign(z)) \mathcal{N}(z; y(w_1 - w_2), 1) dz $$
And rearrange the limits:
$$ = \int^{+\infty}_{0} \mathcal{N}(z; y(w_1 - w_2), 1) dz $$
using $x = y(w_1-w_2) - z$
$$ = \int^{y(w_1 - w_2)}_{-\infty} \mathcal{N}(x; 0, 1) dx $$
$$ = \Phi(y(w_1 - w_2))$$

where $\Phi(a)$ is the gaussian c.d.f, or the *probit* function.

For the probability of player 1 winning, we simply use $p(y=1| w_1, w_2) = p(t>0| w_1, w_2) = \Phi(w_1 - w_2)$

![PerformanceDifference](Figures/PerformanceDifference.jpg)

### Computing the skills (the posterior)

$$p(w_1, w_2 | y) = \frac{Priors \times Likelihood}{Evidence}$$
<br>
$$ = \frac{p(w_1)p(w_2) \times p(y|w_1, w_2)}{\int \int p(w_1)p(w_2) \times p(y|w_1, w_2) dw_1 dw_2}$$
<br>
$$ = \frac{\mathcal{N}(w_1; \mu_1, \sigma_1^2) \mathcal{N}(w_2; \mu_2, \sigma_2^2) \times \Phi(y(w_1 - w_2))}{\int \int \mathcal{N}(w_1; \mu_1, \sigma_1^2) \mathcal{N}(w_2; \mu_2, \sigma_2^2) \times \Phi(y(w_1 - w_2)) dw_1 dw_2}$$

The joint posterior over skills does not have a closed form as the probit function is not closed. Additionally, $w_1$ and $w_2$ have become correlated due to the priors and therefore does not factorise, nor is it a gaussian density function.

Fortunately, the evidence does have a closed form:

$$p(y) = \int \int \mathcal{N}(w_1; \mu_1, \sigma_1^2) \mathcal{N}(w_2; \mu_2, \sigma_2^2) \times \Phi(y(w_1 - w_2)) dw_1 dw_2 = \Phi \bigg(\frac{y(\mu_1 - \mu_2)}{\sqrt{1 + \sigma_1^2 + \sigma_2^2}} \bigg)$$

This is effectively a smoother version of the likelihood as we are using mean skills of each player, and normalising over their variances.

# Load the Data

In [2]:
import scipy.io
mat = scipy.io.loadmat('tennis_data.mat')

In [3]:
PLAYERS = pd.DataFrame(mat['W'], columns=['name']).applymap(lambda x: x[0])  # remove the list around each players name
GAMES = pd.DataFrame(mat['G'], columns=['winner', 'loser'])
display(PLAYERS.head(), GAMES.head())

,name
0,Rafael-Nadal
1,Juan-Monaco
2,Juan-Martin-Del-Potro
3,Mardy-Fish
4,Roger-Federer


,winner,loser
0,1,2
1,1,3
2,1,3
3,1,3
4,1,3


# Gibbs Rank

### Integration of an Intractable Posterior

The basis of Monte-Carlo approximation is:
$$\mathbb{E}_{p(x)} \big[ \Phi(x) \big] \approx \hat{\Phi} = \frac{1}{T} \sum^t_{\tau = 1} \Phi(x^{(\tau)}), \text{  where } x^{(\tau)} \sim p(x)$$

Note that $x^{(\tau)}$ is the $\tau$th d-dimensional sample from the distribution p(x), which is analytically intractable (and typically d>>1).

This is infact an unbiased estimate, with $Var[\hat{\Phi}] = \frac{Var[\Phi]}{T}$. Note $Var[\Phi] = \int (\Phi(x) - \mathbb{E}[\Phi])^2 p(x) dx$. Note that this is independent of dimension d, of x.


How do we generate samples from p(x)? If the distribution has a standard form then we could generate independent samples, however, it is often difficult to sample from this joint distribution if it is within a high dimensional space (the curse of dimensionality). In order to get around this we can use **Gibbs Sampling**, which uses a Markov Chain to generate dependent samples from the desired distribution:

$$x_i' \sim  p(x_i | x_{1, 2, ..., i-1, i+1, ..., d})$$

### Gibbs Sampling

In gibbs sampling, we sample from the proposal distribution, and keep a record of the current state, $x_i$, and the proposal distribution, $q(x^\tau|x^{\tau-1})$. There are several requirements upon this distribution:

1. The markov chain must sample from a known distribution
1. For $\tau \rightarrow \infty$, the distribution $p(x^{(\tau)})$ converges to the required distribution, $p^*(x)$, irrespective of the choice of initial distribution i.e. the markov chain must be ergodic.
1. !!!

Consider the distribution $p(x) = p(x_{1:d})$. The gibbs sampling algorithm is as follows:


> 1. Initialise $\{x_i : i = 1, .., d\}$
> 2. For $\tau = 1, ..., T$: <br>
    > &nbsp;&nbsp;&nbsp;&nbsp; For $i = 1, ..., d$: <br>
        > &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Sample $x_{i}^{\tau+1} \sim p(x_i|x_{\backslash i}^{(\tau)})$




![GibbsSampling](Figures/GibbsSampling.jpg)

Above is an illustration of Gibbs sampmling by alternate updates of two variables, whose distribution is a correlated GAussian. The step size is govened by the standard deviation of the  <span style="color:green">condiditional distribution</span>, and is O(l), leading to slow progression in the direction of elongation of the <span style="color:red">joint distribution</span>. The number of steps needed to obtain an independent sample from the distribution is $O((\frac{L}{l})^2)$. I.e. **Strong correlations slow down gibbs sampling**.

todo. 
1. Read the gibbs section of Pattern Recog and add anything from there into here
1. Reread the notes and add the specific maths
1. implement the gibbs sampling algorithm

$$
\left(\begin{array}{cc} 
0.8944272 & 0.4472136\\
-0.4472136 & -0.8944272
\end{array}\right)
\left(\begin{array}{cc} 
10 & 0\\ 
0 & 5
\end{array}\right)
$$ 

In [5]:
def gaussian_distribution(x, mu, s):
    exponent = -0.5*((x-mu)/s)**2
    Z = (s * np.sqrt(2*np.pi))
    return np.exp(exponent) / Z

In [6]:
def gibbs_sampler(total_iters, players=PLAYERS, games=GAMES):
     skills_container = np.zeros(len(), total_iters)